<a href="https://colab.research.google.com/github/LeonVillanueva/CoLab/blob/master/florante_laura_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku 
import numpy as np

In [0]:
tokenizer = Tokenizer()
data = open('florante_laura.txt').read()

In [0]:
import re
data = re.sub('[.*?]', '', data)

In [0]:
corpus = data.lower().split("\n")

In [0]:
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

In [0]:
input_sequences = []
for line in corpus:
	token_list = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)


In [0]:
# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]

label = ku.to_categorical(label, num_classes=total_words)

In [25]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150, return_sequences = True)))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 11, 100)           336200    
_________________________________________________________________
bidirectional (Bidirectional (None, 11, 300)           301200    
_________________________________________________________________
dropout (Dropout)            (None, 11, 300)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dense (Dense)                (None, 1681)              169781    
_________________________________________________________________
dense_1 (Dense)              (None, 3362)              5654884   
Total params: 6,622,465
Trainable params: 6,622,465
Non-trainable params: 0
______________________________________________

In [0]:
history = model.fit(predictors, label, epochs=100, verbose=1)

Epoch 1/100
287/287 [==============================] - 32s 113ms/step - loss: 7.0032 - accuracy: 0.0474
Epoch 2/100
287/287 [==============================] - 32s 113ms/step - loss: 6.2592 - accuracy: 0.0696
Epoch 3/100
287/287 [==============================] - 32s 113ms/step - loss: 6.0284 - accuracy: 0.0835
Epoch 4/100
 64/287 [=====>........................] - ETA: 24s - loss: 5.9397 - accuracy: 0.0874